## Aprendizado Supervisionado
### prof. Duncan
Prática com Decision Tree Classifier e Iris

In [ ]:
# fixar figuras no conteudo do notebook
%matplotlib inline
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
#
# pacote para separação entre treino e teste
from sklearn.model_selection import train_test_split
#
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
#
# pacotes para validação cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
#
# pacotes de indução de modelos
from sklearn.tree import DecisionTreeClassifier
#
# pacotes para avaliação dos resultados
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.tree import plot_tree, export_graphviz, export_text

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv

#pacotes para visualização e formatação
import pprint
import graphviz
#
# configurações para os diferentes pacotes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# carga de dados
iris = datasets.load_iris(as_frame=True)
print(iris.DESCR)

In [ ]:
# separação em features e target
X = iris.data
y = iris.target
iris.frame

In [ ]:
X.describe()

In [ ]:
iris.frame[['petal length (cm)','petal width (cm)']].plot(kind='scatter',
                                                 x='petal length (cm)',
                                                 y='petal width (cm)',
                                                 s=32, alpha=.8,
                                                 c=y,
                                                 colormap='viridis')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# separação em treino e teste, e X e y

treino_X, teste_X, treino_y, teste_y = train_test_split(X, y, random_state=0,test_size=0.2, stratify=y)

print(treino_X.shape)
print(treino_y.shape)
print(teste_X.shape)
print(teste_y.shape)
print(np.stack(np.unique(teste_y, return_counts=True), axis=1))

## Experimento com reescala de valores, embaralhamento e busca exaustiva nos parâmetros

In [ ]:
# indução do modelo para diferentes valores para os parâmetros, com validação cruzada

#ccp_alphas = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 1.0, 10.0, 100.0]
ccp_alphas = [0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.055, 0.06, 0.07, 0.08, 0.09, 0.1]
#              0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
#ccp_alphas = [0.055]
#ccp_alphas = [0.0]
max_leafs = [2, 3, 4, 5, 6, 7, 8, 9]#, 10, 11, 13, 15, 17, 21]

k_splits = 10

modelo = Pipeline(steps=[('reescala', MinMaxScaler()),
    ('modelo', DecisionTreeClassifier(random_state=0))])

parametros = {'reescala__feature_range':[(0,1)],
              'modelo__ccp_alpha': ccp_alphas,
              'modelo__max_leaf_nodes':max_leafs,
              'modelo__criterion':['gini','entropy']}

skfold = StratifiedKFold(n_splits=k_splits, shuffle=True, random_state=0)

valcruz = GridSearchCV(modelo, parametros,cv=skfold)

valcruz.fit(treino_X, treino_y)

resultados = pd.DataFrame(valcruz.cv_results_).sort_values(by=['rank_test_score','param_modelo__max_leaf_nodes','param_modelo__ccp_alpha' ])
melhor_feature_range = resultados['param_reescala__feature_range'].iloc[0]
melhor_criterion = resultados['param_modelo__criterion'].iloc[0]
melhor_max_leaf_nodes = resultados['param_modelo__max_leaf_nodes'].iloc[0]
melhor_ccp_alpha = resultados['param_modelo__ccp_alpha'].iloc[0]

resultados.head()

In [ ]:
# listar os melhores resultados para os parâmetros

print('Melhor preparação - Feature Range:{}'.format(melhor_feature_range))
print('Melhor resultado - Critério:{}   Máximo número de nodos folha:{}   Alfa:{}    Acurácia média:{:.3f}'.format(
    melhor_criterion,
    melhor_max_leaf_nodes,
    melhor_ccp_alpha,
    resultados['mean_test_score'].iloc[0]))

## Execução para os melhores parâmetros

In [ ]:
escalonador = MinMaxScaler(feature_range = melhor_feature_range)
estimador = DecisionTreeClassifier(ccp_alpha = melhor_ccp_alpha,
                                   max_leaf_nodes = melhor_max_leaf_nodes,
                                   criterion = melhor_criterion)
treino_X_escalonado = escalonador.fit_transform(treino_X, treino_y)
estimador.fit(treino_X_escalonado, treino_y)
teste_X_escalonado = escalonador.transform(teste_X)
teste_pred_y = estimador.predict(teste_X_escalonado)
acuracia = accuracy_score(teste_y, teste_pred_y)
resultado = confusion_matrix(teste_y, teste_pred_y)
cm_display = ConfusionMatrixDisplay(resultado).plot()
fig = cm_display.figure_
fig.set_figheight(3)
fig.set_figwidth(3)
print('Acuracia={:.3f}'.format(acuracia))

In [ ]:
modelo_txt = export_text(estimador, feature_names=iris['feature_names'])
print(modelo_txt)

In [ ]:
# apresentações do modelo
plt.figure(figsize=(10, 10))
plot_tree(estimador, filled=True, rounded=True, feature_names=iris.feature_names, class_names=iris.target_names)
plt.title('Árvore de Decisão treinada no dataset Iris')
plt.show()


## Mostra da distribuição de pontos e classes

In [ ]:
treino_X_escalonado_DF = pd.DataFrame(treino_X_escalonado, columns=list(iris.feature_names))
X_mostra = treino_X_escalonado_DF[['petal length (cm)', 'petal width (cm)']].copy()

teste_X_escalonado_DF = pd.DataFrame(teste_X_escalonado, columns=list(iris.feature_names))
X_teste_mostra = teste_X_escalonado_DF[['petal length (cm)', 'petal width (cm)']].copy()


In [ ]:
modelo_mostra = DecisionTreeClassifier(ccp_alpha = melhor_ccp_alpha,
                                   max_leaf_nodes = melhor_max_leaf_nodes,
                                   criterion = melhor_criterion)

modelo_mostra.fit(X_mostra, treino_y)

In [ ]:
modelo_txt = export_text(modelo_mostra, feature_names=['petal length (cm)', 'petal width (cm)'])
print(modelo_txt)

### Fundo e pontos a partir do treino

In [ ]:
mostra = DecisionBoundaryDisplay.from_estimator(
    modelo_mostra,
    X_mostra,
    response_method="predict",
    cmap=plt.cm.Set3,
    xlabel='petal length (cm)',
    ylabel='petal width (cm)',
)
mostra.ax_.scatter(
    X_mostra['petal length (cm)'], X_mostra['petal width (cm)'], c=treino_y, s=50,
    cmap=plt.cm.viridis, edgecolor="black", linewidth=1
)

### Fundo a partir do treino e pontos a partir do teste

In [ ]:
mostra = DecisionBoundaryDisplay.from_estimator(
    modelo_mostra,
    X_mostra,
    response_method="predict",
    cmap=plt.cm.Set3,
    xlabel='petal length (cm)',
    ylabel='petal width (cm)',
)
mostra.ax_.scatter(
    X_teste_mostra['petal length (cm)'], X_teste_mostra['petal width (cm)'], c=teste_y, s=50,
    cmap=plt.cm.viridis, edgecolor="black", linewidth=1
)